In [1]:
%load_ext autoreload
%autoreload 2
import multiprocessing as mp

from output_check2 import output_check
from synthpop.recipes.starter6_tenure_agehead import Starter
starter_ver='6'

from synthpop.synthesizer import synthesize_all, enable_logging 
import pandas as pd
import os,time,datetime
import pickle

import gc



outputfolder='/home/da/tools/synthpop/demos/outputs/'

# Synthesize by county 

In [3]:
#for whole county
time0=time.time()
print str( datetime.datetime.now())
results=[]

#synthesize_counties(["Livingston County","Oakland County","Washtenaw County","Wayne County","Monroe County","Macomb County","St. Clair County"])
if __name__=='__main__':
    counties=["Wayne County"]
    finalcheck=[]
    for county in counties:
        phases=3
        df_check=None
        for phase in range(phases):
            print "\n county = ",county," phase = ", phase, "\n"
            
            pathout=outputfolder+county+"_starter%s_"%(starter_ver) #output folder
            
            starter = Starter(os.environ["CENSUS"], "MI", county, phase=phase)
           
            households, persons, fit, hh_marginal, pp_marginal = synthesize_all(starter)

            #households.index.name='hh_id' #result[0] is hh
            #households['serialno'] = households['serialno'].map(lambda x: '%f' % x)
            #persons['serialno'] = persons['serialno'].map(lambda x: '%f' % x)
            #results[0].to_csv(outputfolder+county+"_starter%s_"%('3')+"HHs.csv",float_format='%.f') #float_format, not use scentific, pplies to all fp numbers 
            #could use results['serialno'] = results['serialno'].map(lambda x: '%f' % x) to change output format for serialno only
            
            #HHs and Persons output
            if phase == phases-1:
                households.to_csv(pathout+'_households.csv')
                persons.to_csv(pathout+'_persons.csv')


            ##process check tables
            dict_check=output_check({'households':[hh_marginal,households], 'persons':[pp_marginal,persons]})
            #print dict_check.keys() #['households', 'persons', 'hhsize', 'person_factor', 'hhsize7plus']

            outfiles=[pathout+"_check_households.csv",pathout+"_check_households_bycat.csv", \
                      pathout+"_check_persons.csv",pathout+"_check_persons_bycat.csv", \
                      pathout+"_check_hhsize.csv"]

            #set index to check tables and rename columns with phase #
            dfcheck=[dict_check['households'][0],dict_check['households'][1], \
                     dict_check['persons'][0],dict_check['persons'][1], \
                     dict_check['hhsize']]
            indx=['state','county','tract','block group','cat_name']
            indxl=[indx, indx+['cat_value']]*2+[indx+['hh_size']]

            for ind, df in zip(indxl, dfcheck):
                df.set_index(ind, inplace=True)
                df.rename(columns={col:col+"_"+str(phase) for col in df.columns}, inplace=True)

            #join check table from multiple runs            
            if len(finalcheck)==0:
                finalcheck=dfcheck
            else:
                for i in range(5):
                    finalcheck[i]=pd.merge(finalcheck[i], dfcheck[i], left_index=True, right_index=True, how='left')

            #create additional id fields for analysis purpose  
            if phase == phases-1:
                for i in range(5):
                    finalcheck[i].reset_index(inplace=True)
                    finalcheck[i]['bg10']=finalcheck[i]['tract']+finalcheck[i]['block group']
                    finalcheck[i]['geoid10']=finalcheck[i]['state']+finalcheck[i]['county']+ \
                                                finalcheck[i]['tract']+finalcheck[i]['block group']  
                    finalcheck[i].to_csv(outfiles[i])

            if phase == 0:
                dict_check['hhsize7plus'].to_csv("hh7_persons_drawing.csv")
            if phase == 1:
                dict_check['person_factor'].to_csv("hh_size_person_factor.csv")
            if phase == 2:
                os.rename("hh7_persons_drawing.csv", pathout+"_hh7_persons_drawing.csv")
                os.rename("hh_size_person_factor.csv", pathout+"_hh_size_person_factor.csv")

            households,persons,marginals,dict_check = None, None, None, None
            

            with open(pathout+'_fit_stats.pkl', 'wb') as f:
                pickle.dump(fit, f)

            del households,persons,hh_marginal,pp_marginal,dict_check, dfcheck
            
            gc.collect()  
            
print 'total time', time.time()-time0
print str( datetime.datetime.now())

2016-04-20 11:12:09.444495

 county =  Wayne County  phase =  0 

Synthesizing at geog level: 'block_group' (number of geographies is 1822)
memory 1 154173440
multiprocessing time in seconds:  2069.06460905
memory multiprocessing end 6150914048
result produce time in seconds:  2094.909266
memory result end 12515733504

 county =  Wayne County  phase =  1 

Synthesizing at geog level: 'block_group' (number of geographies is 1822)
memory 1 6875922432
multiprocessing time in seconds:  2508.90342498
memory multiprocessing end 12716265472
result produce time in seconds:  2547.23232603
memory result end 18515197952

 county =  Wayne County  phase =  2 

Synthesizing at geog level: 'block_group' (number of geographies is 1822)
memory 1 7297208320
multiprocessing time in seconds:  2117.79833388
memory multiprocessing end 12675231744
result produce time in seconds:  2156.74073601
memory result end 18586238976
total time 7768.42633319
2016-04-20 13:21:37.870831


# Synthesize by selected Block Groups

In [2]:
###for selected BGs
#enable_logging()
time0=time.time()
geoname="problembgs3.csv"
pathout=outputfolder+geoname+"_starter%s_"%(starter_ver)

dfbg=pd.read_csv(geoname,index_col=None,dtype=str) #make sure csv column in text not number, so read as'099' not 99
indd=[ind for i, ind in dfbg.iterrows()]
phases=3
finalcheck=[]
for phase in range(phases):
    print "\nphase = ", phase, "\n"
    starter = Starter(os.environ["CENSUS"], "MI", 'Macomb County', phase=phase)
    households, persons, fit, hh_marginal, pp_marginal = synthesize_all(starter, indexes=indd)
    
    #HHs and Persons output
    if phase == phases-1:
        households.to_csv(pathout+'_households.csv')
        persons.to_csv(pathout+'_persons.csv')
 
    ##process check tables
    dict_check=output_check({'households':[hh_marginal,households], 'persons':[pp_marginal,persons]})
    #print dict_check.keys() #['households', 'persons', 'hhsize', 'person_factor', 'hhsize7plus']
    
    outfiles=[pathout+"_check_households.csv",pathout+"_check_households_bycat.csv", \
              pathout+"_check_persons.csv",pathout+"_check_persons_bycat.csv", \
              pathout+"_check_hhsize.csv"]

    #set index to check tables and rename columns with phase #
    dfcheck=[dict_check['households'][0],dict_check['households'][1], \
             dict_check['persons'][0],dict_check['persons'][1], \
             dict_check['hhsize']]
    indx=['state','county','tract','block group','cat_name']
    indxl=[indx, indx+['cat_value']]*2+[indx+['hh_size']]

    for ind, df in zip(indxl, dfcheck):
        df.set_index(ind, inplace=True)
        df.rename(columns={col:col+"_"+str(phase) for col in df.columns}, inplace=True)

    #join check table from multiple runs            
    if len(finalcheck)==0:
        finalcheck=dfcheck
    else:
        for i in range(5):
            finalcheck[i]=pd.merge(finalcheck[i], dfcheck[i], left_index=True, right_index=True, how='left')
    
    #create additional id fields for analysis purpose  
    if phase == phases-1:
        for i in range(5):
            finalcheck[i].reset_index(inplace=True)
            finalcheck[i]['bg10']=finalcheck[i]['tract']+finalcheck[i]['block group']
            finalcheck[i]['geoid10']=finalcheck[i]['state']+finalcheck[i]['county']+ \
                                        finalcheck[i]['tract']+finalcheck[i]['block group']  
            finalcheck[i].to_csv(outfiles[i])

    if phase == 0:
        dict_check['hhsize7plus'].to_csv("hh7_persons_drawing.csv")
    if phase == 1:
        dict_check['person_factor'].to_csv("hh_size_person_factor.csv")
    if phase == 2:
        os.rename("hh7_persons_drawing.csv", pathout+"_hh7_persons_drawing.csv")
        os.rename("hh_size_person_factor.csv", pathout+"_hh_size_person_factor.csv")
        
    households,persons,marginals,dict_check = None, None, None, None
    
print time.time()-time0


phase =  0 

Synthesizing at geog level: 'block_group' (number of geographies is 627)
memory 1 132161536
multiprocessing time in seconds:  18.5763170719
memory multiprocessing end 151085056
result produce time in seconds: 

/home/da/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:2697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)
output_check2.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_marg_size['persons']=sum_size


 18.6726429462
memory result end 151089152

phase =  1 

Synthesizing at geog level: 'block_group' (number of geographies is 627)
memory 1 145756160


/home/da/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:2705: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)
output_check2.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dfhh7size['hh_size_weight_7']=dfhh7size['persons']/dfhh7size['households']


multiprocessing time in seconds:  19.5994870663
memory multiprocessing end 164040704
result produce time in seconds:  19.6538219452
memory result end 164040704

phase =  2 

Synthesizing at geog level: 'block_group' (number of geographies is 627)
memory 1 150106112
multiprocessing time in seconds:  20.3050680161
memory multiprocessing end 163823616
result produce time in seconds:  20.3579189777
memory result end 163823616
76.2619900703


In [ ]:
chisquare([18, 7, 42, 33],[15, 10, 45, 30],1)

In [ ]:
from synthpop.draw import compare_to_constraints 

In [ ]:
a=pd.Series([18, 7, 42, 33,33])

In [ ]:
b=pd.Series([15, 15, 10, 45, 30])

In [ ]:
compare_to_constraints(a,b)

In [ ]:
b.value_counts()

In [ ]:
a.align(b)

In [ ]:
a

In [ ]:
b

In [ ]:
if a_list:
    print 1

In [ ]:
a[b>=15]

In [ ]:
import numpy as np
arrays = [['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
       ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]
tuples = list(zip(*arrays))
index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])
s = pd.Series(np.random.randn(8),index=index)
s=s.to_frame(name='col1')

In [ ]:
s['col2']=s['col1']+2

In [ ]:
for name, group in s.groupby(level=range(2)):
    print name
    print type(group['col1'])

In [ ]:
chisquare([16, 18, 16, 14, 12, 12], f_exp=[16, 16, 16, 16, 16, 8],ddof=1)

In [ ]:
chisquare([18, 7, 42,33], f_exp=[15, 10, 45, 30])

In [ ]:
dk=pd.DataFrame.from_dict({'a':[1 ,2],'b':[3,4] },orient='index')

In [ ]:
print dk.index.name

In [ ]:
a={'households':[1,2], 'persons':[3,4]}

In [ ]:
for k,v in a.keys(),a.values():
    print k
    print v

In [ ]:
a.keys()

In [ ]:
b.keys

In [ ]:
b={}

In [ ]:
data = {'Column 1'     : [1., 2., 3., 4.],
        'Index Title'  : ["Apples", "Oranges", "Puppies", "Ducks"]}
df = pd.DataFrame(data)

In [ ]:
df.index.name

In [ ]:
import numpy as np
import pandas as pd
df = pd.DataFrame({'A': ['foo', 'bar', 'foo', 'bar',
                         'foo', 'bar', 'foo', 'foo'],
                   'B': ['one', 'one', 'two', 'three',
                         'two', 'two', 'one', 'three'],
                   'number A': np.arange(8),
                   'number B': np.arange(8) * 2})
grouped = df.groupby('A')

print(grouped.agg({
    'number A': 'sum',
    'number B': 'min'}))

In [ ]:
print type(grouped.agg({
    'number A': 'sum',
    'number B': 'min'}))

In [ ]:
grouped.agg({
    'number A': 'sum',
    'number B': 'min'})

In [ ]:
cat[]=0

In [ ]:
cat=[0]

In [ ]:
cat.append(1)

In [ ]:
cat

In [ ]:
for i in [*[1,2],*[3,4],5]:
    print i

In [ ]:
if a_list:
    print 'no'

In [ ]:
d={}

In [ ]:
[[2],[3]+[4]]*2